In [16]:

import tensorflow as tf

import keras



#OLD IMPORT
#from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell
#mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

#NEW import
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#NOT USED ANYMORE
#cast_x_train = tf.cast(x_train, 'float32')
#cast_y_train = tf.cast(y_train, 'float32')
#cast_x_test = tf.cast(x_test, 'float32')
#cast_y_test = tf.cast(y_test, 'float32')

In [17]:
hm_epochs = 10
n_classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28
rnn_size = 128

tf.compat.v1.disable_eager_execution()

#x=1234
x = tf.compat.v1.placeholder('float', [None, n_chunks, chunk_size])
y = tf.compat.v1.placeholder('float')

ein neurales netzwerk - spiegeln?

In [18]:
def recurrent_neural_network(x):
    #tf.compat.v1.reset_default_graph()    
    layer = {'weights':tf.Variable(tf.random.normal([rnn_size, n_classes])),
            'biases':tf.Variable(tf.random.normal([n_classes]))}
    #print("inner x: " + str(x))

    x = tf.transpose(a=x, perm=[1, 0, 2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)

    lstm_cell = tf.keras.layers.LSTMCell(rnn_size)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    output = tf.matmul(outputs[-1], layer['weights']) + layer['biases']

    return output

In [19]:
def train_neural_network(x):
    prediction = recurrent_neural_network(x)

    #print(type(prediction))
    #print(prediction.shape)
    #print(cast_y_train.shape)

    cost = tf.reduce_mean(input_tensor=tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=tf.stop_gradient(y)))
    optimizer = tf.compat.v1.train.AdamOptimizer().minimize(cost)

    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(len(x_train)/batch_size)):
                epoch_x, epoch_y = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))

                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed oput of', hm_epochs, 'loss:', epoch_loss)

    correct = tf.equal(tf.argmax(input=prediction, axis=1), tf.argmax(input=y, axis=1))

    accuracy = tf.reduce_mean(input_tensor=tf.cast(correct, 'float'))
    print('Accuracy:', accuracy.eval({x_test.images.reshape((-1, n_chunks, chunk_size)), y_test.labels}))

In [20]:
train_neural_network(x)

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Use `tf.global_variables_initializer` instead.


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [ ]:
#test = tf.convert_to_tensor(x_train, dtype='float32')
#test2 = tf.cast(test, 'float32')

#cast_x_train = tf.cast(x_train, 'float32')
#cast_y_train = tf.cast(y_train, 'float32')
#cast_x_test = tf.cast(x_test, 'float32')
#cast_y_test = tf.cast(y_test, 'float32')




#cast doesnt change shape
#print(x_train.shape)
#print(cast_x_train.shape)

#print(y_train.shape)
#print(cast_y_train.shape)

In [ ]:
train_neural_network(x)